In [1]:
import pandas as pd

In [2]:
from transform_ps.etl.data_type import DataFactory
from transform_ps.etl.util import clear_cwd_cache

In [3]:
clear_cwd_cache()

cache is now empty


In [4]:
factory = DataFactory()

### Get Subscriber info

In [5]:
sub = factory.create_data_type('subscriber')

In [6]:
sub.get_data(fys=20)

Finished prepare_data in 0.0096 secs
Finished prepare_data in 0.0117 secs


In [7]:
clx_pkgs = sub.working.loc[sub.working.season_desc.map(lambda x: 'Classics' in x)].reset_index(drop=True)

In [8]:
thu_sat_clx = clx_pkgs.loc[clx_pkgs.pkg_desc.map(lambda x: 'Thu' in x or 'Sat' in x)]

In [9]:
thu_sat_clx_subs = list(thu_sat_clx.customer_no.drop_duplicates())

### Get Donor Info

In [10]:
donor = factory.create_data_type('donor')

In [11]:
donor.get_data(fys=13)

not found in cache
saved working to cache
Finished prepare_data in 2.4305 secs


### Filter for Annual Fund

* Green - Leave on the list and count toward giving total (should be all the funds Nikki said were ok)
* Yellow - Leave on the list but do not count toward giving total
* Red - Remove from list and do not count toward giving total

In [12]:
fund_categories = {row[0]: row[1] for index, row in pd.read_csv('funds_fy19-20.csv').iterrows()}

##### FY20 Donors for manip

In [13]:
fy20_donors = donor.type.filters['fys'](donor.working, fys=[20])
fy20_donors['fund_color_cat'] = fy20_donors['fund_desc'].map(fund_categories)

##### FY19-20 Donors for manip

In [14]:
fy19_20_donors = donor.type.filters['fys'](donor.working, fys=[19, 20])
fy19_20_donors['fund_color_cat'] = fy19_20_donors['fund_desc'].map(fund_categories)

##### FY19 and FY20 donors separated for working with

In [15]:
d19 = donor.type.filters['fys'](donor.working, fys=[19])
d20 = donor.type.filters['fys'](donor.working, fys=[20])

In [16]:
d19['fund_color_cat'] = d19['fund_desc'].map(fund_categories)
d20['fund_color_cat'] = d20['fund_desc'].map(fund_categories)

##### FY20 Box Circle Members (Classics/Pops) + Red Donors

In [17]:
fy20_box = fy20_donors.loc[fy20_donors['fund_color_cat'] == 'Box']['summary_cust_id'].drop_duplicates()

In [18]:
red_donors = fy19_20_donors.loc[fy19_20_donors['fund_color_cat'] == 'Red']['summary_cust_id'].drop_duplicates()

#### Filter out all customers in red_donors and fy20_box (customers)

In [19]:
d19 = d19.loc[~d19['summary_cust_id'].isin(fy20_box)]
d20 = d20.loc[~d20['summary_cust_id'].isin(fy20_box)]

In [20]:
d19 = d19.loc[~d19['summary_cust_id'].isin(red_donors)]
d20 = d20.loc[~d20['summary_cust_id'].isin(red_donors)]

#### Filter out yellow funds (not customers)

In [21]:
d19 = d19.loc[d19.fund_color_cat != 'Yellow']
d20 = d20.loc[d20.fund_color_cat != 'Yellow']

#### Aggregate donor totals

In [22]:
d19_agg = d19.groupby('customer_no').agg({'gift_plus_pledge': 'sum'}).reset_index()
d20_agg = d20.groupby('customer_no').agg({'gift_plus_pledge': 'sum'}).reset_index()

#### Filter out >=1000 and < 2500

In [23]:
d19_identified = d19_agg.loc[d19_agg['gift_plus_pledge'].map(lambda x: x >= 1000 and x < 2500)].reset_index(drop=True)
d20_identified = d20_agg.loc[d20_agg['gift_plus_pledge'].map(lambda x: x >= 1000 and x < 2500)].reset_index(drop=True)

In [24]:
d19_identified.loc[d19_identified['customer_no'].isin(thu_sat_clx_subs)].reset_index(drop=True).to_excel('d19.xlsx')

In [25]:
d20_identified.loc[d20_identified['customer_no'].isin(thu_sat_clx_subs)].reset_index(drop=True).to_excel('d20.xlsx')